In [24]:
example = [
'.......S.......',
'...............',
'.......^.......',
'...............',
'......^.^......',
'...............',
'.....^.^.^.....',
'...............',
'....^.^...^....',
'...............',
'...^.^...^.^...',
'...............',
'..^...^.....^..',
'...............',
'.^.^.^.^.^...^.',
'...............'
]

In [25]:
split_indices = set()
def splits(input, start_row, beam_start_index):
  for i, row in enumerate(input):
    if i >= start_row and row[beam_start_index] == '^':
      split_indices.add((i, beam_start_index))
      splits(input, i+1, beam_start_index-1)
      splits(input, i+1, beam_start_index+1)
      break

In [26]:
splits(example[1:], 0, example[0].index('S'))
len(split_indices)

21

In [28]:
# split_indices = set()
with open('input.txt', 'r') as file:
    input = file.read().split('\n')

# splits(input[1:], 0, input[0].index('S'))
# len(split_indices)

Initially made a recursive function, but much faster to do queue/stack

In [32]:
from collections import deque

def count_splits_faster(grid):
    rows = len(grid)
    cols = len(grid[0])
    split_indices = set()
    
    # start with the beam at row 0, column of 'S'
    start_col = grid[0].index('S')
    stack = deque([(1, start_col)])  # start from row 1
    
    while stack:
        r, c = stack.pop()
        if r >= rows or c < 0 or c >= cols:
            continue
        if grid[r][c] == '^':
            if (r, c) not in split_indices:
                split_indices.add((r, c))
                # new beams go down-left and down-right from the next row
                stack.append((r+1, c-1))
                stack.append((r+1, c+1))
        else:
            # empty space, continue downward
            stack.append((r+1, c))
    
    return len(split_indices)


In [34]:
count_splits_faster(example)

21

In [33]:
count_splits_faster(input)

1633

# Part 2:

DYNAMIC PROGRAMMING

If we define dp[r][c] = number of timelines that reach cell (r, c), we can solve this bottom-up or top-down with dynamic programming.

Initialize dp[0][S_col] = 1 (the starting beam)

Iterate rows from top to bottom:

If cell is empty ., propagate timelines downward:

dp[r+1][c] += dp[r][c]


If cell is a splitter ^, propagate timelines left and right:

dp[r+1][c-1] += dp[r][c]
dp[r+1][c+1] += dp[r][c]


The total number of timelines is the sum of all dp values at the bottom row (or you can sum all non-zero dp[r][c] if you just want the total number of timelines).

You process each cell once, O(rows × cols)


In [40]:
def count_timelines(grid):
    rows = len(grid)
    cols = len(grid[0])
    
    dp = [[0]*cols for _ in range(rows)]
    
    start_col = grid[0].index('S')   # include row 0!
    dp[0][start_col] = 1
    
    for r in range(rows):
        for c in range(cols):
            if dp[r][c] == 0:
                continue
            if grid[r][c] == '.':
                if r+1 < rows:
                    dp[r+1][c] += dp[r][c]
            elif grid[r][c] == '^':
                if r+1 < rows:
                    if c-1 >= 0:
                        dp[r+1][c-1] += dp[r][c]
                    if c+1 < cols:
                        dp[r+1][c+1] += dp[r][c]
            else:  # for 'S' or any other symbol
                if r+1 < rows:
                    dp[r+1][c] += dp[r][c]
    
    return sum(dp[-1])


In [41]:
count_timelines(example)

40

In [42]:
count_timelines(input)

34339203133559